## Zhlukovanie - vyhodnotenie a interpretácia zhlukov - príklad 2

Rovnakýn postup ako v predošlom príklade si vieme demonštrovať aj na inom príklade. V tomto datasete budeme pracovať s dátami, ktoré charakterizujú zákazníkov. Každý z nich je popísaný:
* Gender - pohlavie
* Age - vek
* Annual Income - ročný príjem v dolároch
* Spending Score - index popisujúci tendenciu zákazníka nakupovať

In [ ]:
# importujeme potrebné knižnice

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# načítame dáta do dátového rámca a vypíšeme prvých 5 riadkov

data = pd.read_csv('../data/customers.csv')
data.head()

Najprv si dáta pripravíme jednoduchým predspracovaním:
* odstránime identifikátor zákazníka
* zakódujeme atribút pohlavie

In [ ]:
# YOUR CODE HERE

Rovnako ako v predošlom príklade identifikujeme vhodný počet zhlukov a natrénujeme model K-Means pre zvolený počet. Skúsime použiť obe kritériá  - aj `inertia` aj `silhouette` pre rozhodnutie. 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

results_inertia = []
results_silhouette = []

K = range(2,10)

for k in K:
    model = KMeans(n_clusters=k)
    model.fit(data)
    predictions = model.predict(data)
    results_inertia.append(model.inertia_)
    results_silhouette.append(silhouette_score(data, predictions))
    
print("Skore Inertia:")
print(results_inertia)
print("Skore Silhouette:")
print(results_silhouette)

Aby sme mali lepší prehľad, môžeme si oba grafy vykresliť. Pomocou `subplot()` vieme vykresliť oba grafy súčasne vedľa seba.

In [ ]:
plt.figure(figsize=(12, 4)) # definujeme veľkosť obrázku (trocha natiahnuť do šírky, kvôli vykresleniu 2 vedľa seba)
plt.subplot(1, 2, 1) # 1-2-1 znamená, že vytvoríme 1 riadok, 2 stĺpce a kresliť budeme do 1.
plt.plot(K, results_inertia, 'bx-')
plt.xlabel('Pocet zhlukov')
plt.ylabel('Suma stvorcov vzdialenosti')
plt.title('Inertia')

plt.subplot(1, 2, 2) # 1-2-2 znamená, že vytvoríme 1 riadok, 2 stĺpce a kresliť budeme do 2.
plt.plot(K, results_silhouette, 'bx-')
plt.xlabel('Pocet zhlukov')
plt.ylabel('Silhouette')
plt.title('Silhouette')

plt.show() # show funkciu zavoláme až nakoniec, čím vykreslíme oba grafy naraz

Ideálny počet zhlukov sme odhalili a teraz natrénujeme K-Means model s daným počtom zhlukov. POužijeme natrénovaný model a príklady zatriedime do zhlukov. Vytvorenú premennú tak použijeme pre klasifikačný model pre popis zhlukov.

In [ ]:
model = KMeans(n_clusters=6)
model.fit(data)
labels = model.predict(data)

In [ ]:
# vypíšeme predikcie modelu

print(labels)

Teraz natrénujeme klasifikátor, ktorým zhluky popíšeme.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

dt = DecisionTreeClassifier(max_depth=4)   # Inicializácia stromového modelu   
dt.fit(data, labels)        # Trénovanie modelu na trénovacej množine 

y_dt = dt.predict(data)       # Otestovanie modelu na testovacej množine

from sklearn.metrics import accuracy_score,precision_score, recall_score # vypočítanie metrík kvality modelu

print(f"Presnosť (accuracy) modelu: {accuracy_score(labels, y_dt)}")

cm = confusion_matrix(labels, y_dt)  # vypísanie kontigenčnej tabuľky
print(cm)

Stromový model vykreslíme pre odhadnutie pravidiel pre dané zhluky. Pokúsime sa odvodiť pravidlá pre popis jednotlivých zhlukov.

In [ ]:
from sklearn import tree
from sklearn.tree import export_graphviz

with open("decision_tree.txt", "w") as f:
    f = tree.export_graphviz(dt, feature_names=data.columns.values, class_names=["0","1","2","3","4","5"], out_file=f)

### Úloha 13.6:

Pokúste sa pre identifikované zhluky vytvoriť aj vizualizácie priemerných hodnôt ich číselných atribútov. Porovnajte tieto s extrahovanými pravidlami, resp. so znalosťami viditeľnými zo stromovej štruktúry modelu. 

In [ ]:
# YOUR CODE HERE